In [24]:
%matplotlib inline

import pickle as pkl
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [25]:
# Download mnist data

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
# Create model inputs

def model_inputs(discriminator_dim, generator_dim):
    discriminator_inputs = tf.placeholder(
        tf.float32,
        shape=[None, discriminator_dim],
        name='discriminator_input'
    )
    generator_inputs = tf.placeholder(
        tf.float32,
        shape=[None, generator_dim],
        name='generator_input'
    )
    return discriminator_inputs, generator_inputs

In [27]:
# Create generator

def generator(_input, out_dim, num_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(_input, num_units)
        leaky_relu = tf.maximum(hidden_layer * alpha, hidden_layer)
        
        logits = tf.layers.dense(leaky_relu, out_dim)
        return tf.tanh(logits)

In [28]:
# Create discriminator

def discriminator(_input, num_units=128, reuse=False, alpha=0.01):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(_input, num_units)
        leaky_relu = tf.maximum(hidden_layer * alpha, hidden_layer)
        
        logits = tf.layers.dense(leaky_relu, 1)
        return tf.sigmoid(logits), logits

In [29]:
# Set hyperparameters

discriminator_input_size = 728
generator_input_size = 100

generator_hidden_size = discriminator_hidden_size = 128

alpha = 0.01


smooth = 0.1

In [32]:
# Building the network

tf.reset_default_graph()
## Create input placeholders
discriminator_input, generator_input = model_inputs(
    discriminator_input_size, generator_input_size)


## Build the model

generator_model = generator(generator_input, discriminator_input_size)

discriminator_model_real, discriminator_logits_real = discriminator(discriminator_input)
discriminator_model_fake, discriminator_logits_fake = discriminator(generator_model, reuse=True)

In [ ]:
# Losses

## Discriminator loss real

discriminator_loss_real = tf.nn.sigmoid_cross_entropy_with_logits(
    logits=discriminator_logits_real,
    labels=tf.ones_like(discriminator_logits_real) * (1 - smooth)
)
discriminator_loss_real = tf.reduce_mean(discriminator_loss_real)

## Discriminator loss fake

discriminator_loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(
    logits=discriminator_logits_fake,
    labels=tf.zeros_like(discriminator_logits_real)
)
discriminator_loss_fake = tf.reduce_mean(discriminator_loss_fake)

## Discriminator loss

discriminator_loss = discriminator_loss_real + discriminator_loss_fake

## Generator loss

generator_loss = tf.nn.os